## Setup

In [1]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [2]:
load_eoepca_state()

In [3]:
platform_domain = os.environ.get("INGRESS_HOST")
resource_registration_domain = f'{os.environ.get("HTTP_SCHEME")}://registration-api.{platform_domain}'

print(f"Resource Registration API: {resource_registration_domain}")

Resource Registration API: https://registration-api.notebook-test.develop.eoepca.org


## Validate Resource Registration API Endpoints

In [5]:
endpoints = [
    ("Landing Page", resource_registration_domain),
    ("Swagger UI", f"{resource_registration_domain}/openapi?f=html"),
    ("Processes Endpoint", f"{resource_registration_domain}/processes"),
    ("Jobs Endpoint", f"{resource_registration_domain}/jobs")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

Landing Page (https://registration-api.notebook-test.develop.eoepca.org): 200
Swagger UI (https://registration-api.notebook-test.develop.eoepca.org/openapi?f=html): 200
Processes Endpoint (https://registration-api.notebook-test.develop.eoepca.org/processes): 200
Jobs Endpoint (https://registration-api.notebook-test.develop.eoepca.org/jobs): 200


## Testing a Simple Hello-World Process Execution

In [6]:
hello_world_url = f"{resource_registration_domain}/processes/hello-world/execution"

payload = {
    "inputs": {
        "name": "EOEPCA Tester",
        "message": "Validation of Hello-World Process"
    }
}

response = requests.post(hello_world_url, json=payload)

print(f"Hello-World execution status code: {response.status_code}")
print(response.json())

Hello-World execution status code: 200
{'id': 'echo', 'value': 'Hello EOEPCA Tester! Validation of Hello-World Process'}


## Registering a Sample Dataset

### Prepare Sample STAC Item

In [7]:
sample_stac_item_path = Path("simple-item.json")

if not sample_stac_item_path.exists():
    stac_item_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/scripts/resource-registration/data/simple-item.json"
    response = requests.get(stac_item_url)
    sample_stac_item_path.write_text(response.text)

print(sample_stac_item_path.read_text()[:500])

{
    "stac_version": "1.1.0",
    "stac_extensions": [],
    "type": "Feature",
    "id": "20201211_223832_CS2",
    "bbox": [
      172.91173669923782,
      1.3438851951615003,
      172.95469614953714,
      1.3690476620161975
    ],
    "geometry": {
      "type": "Polygon",
      "coordinates": [
        [
          [
            172.91173669923782,
            1.3438851951615003
          ],
          [
            172.95469614953714,
            1.3438851951615003
          ],
          


### Execute Resource Registration

In [8]:
register_url = f"{resource_registration_domain}/processes/register/execution"

payload = {
    "inputs": {
        "type": "dataset",
        "source": stac_item_url,
        "target": f"https://resource-catalogue.{platform_domain}/stac"
    }
}

response = requests.post(register_url, json=payload)

print(f"Dataset registration status code: {response.status_code}")
registration_result = response.json()
registration_result

Dataset registration status code: 200


{'id': 'registrar',
 'resource-and-data-catalogue-link': {'href': 'https://resource-catalogue.notebook-test.develop.eoepca.org/stac/collections/metadata:main/items/20201211_223832_CS2',
  'rel': 'item',
  'type': 'application/geo+json'}}

## Verify Registration Job Status

In [9]:
jobs_url = f"{resource_registration_domain}/jobs"
response = requests.get(jobs_url)
jobs = response.json()

print(f"Jobs listing status code: {response.status_code}")
latest_job = jobs["jobs"][0] if jobs.get("jobs") else {}
print(json.dumps(latest_job, indent=2))

Jobs listing status code: 200
{
  "type": "process",
  "processID": "register",
  "jobID": "15884148-066d-11f0-9103-422a2051c4ad",
  "status": "successful",
  "message": "Job complete",
  "progress": 100,
  "parameters": null,
  "job_start_datetime": "2025-03-21T15:56:43.093950Z",
  "job_end_datetime": "2025-03-21T15:56:47.093965Z",
  "links": [
    {
      "href": "https://registration-api.notebook-test.develop.eoepca.org/jobs/15884148-066d-11f0-9103-422a2051c4ad/results?f=html",
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/results",
      "type": "text/html",
      "title": "Results of job as HTML"
    },
    {
      "href": "https://registration-api.notebook-test.develop.eoepca.org/jobs/15884148-066d-11f0-9103-422a2051c4ad/results?f=json",
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/results",
      "type": "application/json",
      "title": "Results of job as JSON"
    }
  ]
}


## Validate Registered Dataset in Resource Discovery

In [10]:
resource_discovery_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}'
metadata_items_url = f"{resource_discovery_domain}/collections/metadata:main/items"

response = requests.get(metadata_items_url)
items = response.json().get("features", [])

item_ids = [item.get("id") for item in items]
expected_item_id = json.loads(sample_stac_item_path.read_text()).get("id")

if expected_item_id in item_ids:
    print(f"✅ Dataset '{expected_item_id}' successfully registered and discoverable!")
else:
    print(f"❌ Dataset '{expected_item_id}' registration failed or item not discoverable.")

✅ Dataset '20201211_223832_CS2' successfully registered and discoverable!


## Additional Validation - Flowable API Check

In [11]:
flowable_domain = f'{os.environ.get("HTTP_SCHEME")}://registration-harvester-api.{platform_domain}'
flowable_docs_url = f"{flowable_domain}/flowable-rest/docs"
response = requests.get(flowable_docs_url)

print(f"Flowable Swagger UI status code: {response.status_code}")
response.url

Flowable Swagger UI status code: 200


'https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/docs/'

## List Flowable Deployed Processes

In [12]:
flowable_admin_user = os.environ.get("FLOWABLE_ADMIN_USER", "eoepca")
flowable_admin_password = os.environ.get("FLOWABLE_ADMIN_PASSWORD", "eoepca")

process_definitions_url = f"{flowable_domain}/flowable-rest/service/repository/process-definitions"

response = requests.get(process_definitions_url, auth=(flowable_admin_user, flowable_admin_password))

print(f"Flowable process definitions status code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Flowable process definitions status code: 200
{
  "data": [
    {
      "id": "createTimersProcess:1:641b2360-066c-11f0-9621-dac2883c12fb",
      "url": "https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/process-definitions/createTimersProcess:1:641b2360-066c-11f0-9621-dac2883c12fb",
      "key": "createTimersProcess",
      "version": 1,
      "name": "Create timers process",
      "description": "Test process to create a number of timers.",
      "tenantId": "",
      "deploymentId": "636540d0-066c-11f0-9621-dac2883c12fb",
      "deploymentUrl": "https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/deployments/636540d0-066c-11f0-9621-dac2883c12fb",
      "resource": "https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/deployments/636540d0-066c-11f0-9621-dac2883c12fb/resources/createTimersProcess.bpmn20.xml",
      "diagramResource": null,